In [33]:
import torch
import torchtext
import time

from torchtext import datasets

import numpy as np
import torch.nn as nn
import torch.nn.functional as functional


In [34]:
start = time.time()
TEXT = torchtext.data.Field(sequential=True, batch_first=True, lower=True)
LABEL = torchtext.data.Field(sequential=False, batch_first=True)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(split_ratio=0.8)

TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

BATCH_SIZE = 100
device = torch.device("CUDA" if torch.cuda.is_available() else "cpu")

In [35]:
train_iter, valid_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device
)

In [36]:
vocab_size = len(TEXT.vocab)
n_classes = 2

In [37]:
class BasicRNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicRNN, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers=n_layers, batch_first=True)
        self.out = nn.Linear(hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.rnn(x, h_0)
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [38]:
model = BasicRNN(
    n_layers=1,
    hidden_dim=256,
    n_vocab=vocab_size,
    embed_dim=128,
    n_classes=n_classes,
    dropout_p=0.5,
)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [39]:
def train(epoch, model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        optimizer.zero_grad()

        logit = model(x)
        loss = functional.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        if b % 50 == 0:
            print(
                f"Train Epoch: {epoch} [{b * len(x)}/{len(train_iter.dataset)} ", end=""
            )
            print(f"({len(train_iter.dataset):.0f}%)]\tLoss: {loss.item():.6f}")

In [40]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total, total_loss = 0, 0, 0

    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        logit = model(x)
        loss = functional.cross_entropy(logit, y, reduction="sum")
        total += y.size(0)
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [41]:
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 5

for e in range(1, EPOCHS + 1):
    train(e, model, optimizer, train_iter)
    valid_loss, valid_accuracy = evaluate(model, valid_iter)
    print(f"\n[epoch: {e:3d}] Validation_loss: {valid_loss:4.2f}", end="")
    print(f"| Validation_acc: {valid_accuracy:4.2f}")

Train Epoch: 1 [0/20000 (20000%)]	Loss: 0.697548
Train Epoch: 1 [5000/20000 (20000%)]	Loss: 0.708910
Train Epoch: 1 [10000/20000 (20000%)]	Loss: 0.697511
Train Epoch: 1 [15000/20000 (20000%)]	Loss: 0.693477

[epoch:   1] Validation_loss: 0.70| Validation_acc: 0.49
Train Epoch: 2 [0/20000 (20000%)]	Loss: 0.693898
Train Epoch: 2 [5000/20000 (20000%)]	Loss: 0.694291
Train Epoch: 2 [10000/20000 (20000%)]	Loss: 0.694285
Train Epoch: 2 [15000/20000 (20000%)]	Loss: 0.702581

[epoch:   2] Validation_loss: 0.69| Validation_acc: 0.50
Train Epoch: 3 [0/20000 (20000%)]	Loss: 0.688070
Train Epoch: 3 [5000/20000 (20000%)]	Loss: 0.704844
Train Epoch: 3 [10000/20000 (20000%)]	Loss: 0.693793
Train Epoch: 3 [15000/20000 (20000%)]	Loss: 0.695230

[epoch:   3] Validation_loss: 0.70| Validation_acc: 0.50
Train Epoch: 4 [0/20000 (20000%)]	Loss: 0.702174
Train Epoch: 4 [5000/20000 (20000%)]	Loss: 0.695602
Train Epoch: 4 [10000/20000 (20000%)]	Loss: 0.688003
Train Epoch: 4 [15000/20000 (20000%)]	Loss: 0.69315